## Let's implement CCS from scratch.
This will deliberately be a simple (but less efficient) implementation to make everything as clear as possible.


links:
- [loading](https://github.com/deep-diver/LLM-As-Chatbot/blob/main/models/alpaca.py)
- [dict](https://github.com/deep-diver/LLM-As-Chatbot/blob/c79e855a492a968b54bac223e66dc9db448d6eba/model_cards.json#L143)
- [prompt_format](https://github.com/deep-diver/PingPong/blob/main/src/pingpong/alpaca.py)

In [1]:

import copy
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from typing import Optional, List, Dict, Union

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from torch import optim
from torch.utils.data import random_split, DataLoader, TensorDataset

import pickle
import hashlib
from pathlib import Path

from datasets import load_dataset
import datasets

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForMaskedLM, AutoModelForCausalLM, AutoConfig
import transformers
from transformers.models.auto.modeling_auto import AutoModel
from transformers import LogitsProcessorList


import lightning.pytorch as pl
from dataclasses import dataclass

from sklearn.linear_model import LogisticRegression
# from scipy.stats import zscore
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from sklearn.preprocessing import RobustScaler

from tqdm.auto import tqdm
import gc
import os

from loguru import logger
logger.add(os.sys.stderr, format="{time} {level} {message}", level="INFO")


transformers.__version__

'4.30.1'

# Model

Chosing:
- https://old.reddit.com/r/LocalLLaMA/wiki/models
- https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard
- https://github.com/deep-diver/LLM-As-Chatbot/blob/main/model_cards.json


A uncensored and large one might be best for lying.

In [2]:
from peft import PeftModel


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/ubuntu/mambaforge/envs/dlk2/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/ubuntu/mambaforge/envs/dlk2/lib/libcudart.so.11.0'), PosixPath('/home/ubuntu/mambaforge/envs/dlk2/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


In [3]:
# leaderboard https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard
model_options = dict(
    device_map="auto", 
    # load_in_4bit=True,
    load_in_8bit=True,
    # torch_dtype=torch.float16,
    trust_remote_code=True,
    # use_cache=False,
)

# so I need to use either pythia, stablelm, or tiiuae/falcon-7b-instruct to get dropout...
# moel_repo = "stabilityai/stablelm-tuned-alpha-7b" # poor performance

# https://github.com/deep-diver/LLM-As-Chatbot/blob/main/models/falcon.py
model_repo = "tiiuae/falcon-7b-instruct"
# model_repo = "togethercomputer/RedPajama-INCITE-7B-Instruct"
# model_repo = "OpenAssistant/oasst-sft-4-pythia-12b-epoch-3.5"
# model_repo = "TheBloke/OpenAssistant-SFT-7-Llama-30B-HF"
model_repo = "OpenAssistant/falcon-7b-sft-top1-696" # too "safe" to even answer
# model_repo = "OpenAssistant/falcon-7b-sft-mix-2000"
lora_repo = None

# wizardlm format
# model_repo = "ehartford/WizardLM-Uncensored-Falcon-7b"

# # # # uses Vicuna format https://huggingface.co/junelee/wizard-vicuna-13b/discussions/1
# model_repo = "TheBloke/Wizard-Vicuna-13B-Uncensored-HF"
# lora_repo = None

# model_repo = "openaccess-ai-collective/manticore-13b"
# lora_repo = None

# model_repo = "elinas/llama-13b-hf-transformers-4.29" # no dropout
# # lora_repo = "LLMs/AlpacaGPT4-LoRA-13B-elina"

# model_repo = "ehartford/Wizard-Vicuna-13B-Uncensored"
# model_repo = "dvruette/llama-13b-pretrained-dropout"

config = AutoConfig.from_pretrained(model_repo, trust_remote_code=True,)
print(config)
config.hidden_dropout=0.2
config.attention_dropout=0.2
config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(model_repo)
model = AutoModelForCausalLM.from_pretrained(model_repo, config=config, **model_options)

if lora_repo is not None:
    # https://github.com/tloen/alpaca-lora/blob/main/generate.py#L40
    from peft import PeftModel
    model = PeftModel.from_pretrained(
        model,
        lora_repo, 
        torch_dtype=torch.bfloat16,
        lora_dropout=0.2,
        device_map='auto'
    )
    
# model = model.half()

RWConfig {
  "_name_or_path": "OpenAssistant/falcon-7b-sft-top1-696",
  "alibi": false,
  "apply_residual_connection_post_layernorm": false,
  "architectures": [
    "RWForCausalLM"
  ],
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "OpenAssistant/falcon-7b-sft-top1-696--configuration_RW.RWConfig",
    "AutoModel": "OpenAssistant/falcon-7b-sft-top1-696--modelling_RW.RWModel",
    "AutoModelForCausalLM": "OpenAssistant/falcon-7b-sft-top1-696--modelling_RW.RWForCausalLM",
    "AutoModelForQuestionAnswering": "OpenAssistant/falcon-7b-sft-top1-696--modelling_RW.RWForQuestionAnswering",
    "AutoModelForSequenceClassification": "OpenAssistant/falcon-7b-sft-top1-696--modelling_RW.RWForSequenceClassification",
    "AutoModelForTokenClassification": "OpenAssistant/falcon-7b-sft-top1-696--modelling_RW.RWForTokenClassification"
  },
  "bias": false,
  "bos_token_id": 11,
  "eos_token_id": 11,
  "hidden_dropout": 0.0,
  "hidden_size": 4544,
  "initializer_range": 0.02,
  "layer_no

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [4]:
model

RWForCausalLM(
  (transformer): RWModel(
    (word_embeddings): Embedding(65040, 4544)
    (h): ModuleList(
      (0-31): 32 x DecoderLayer(
        (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
        (self_attention): Attention(
          (maybe_rotary): RotaryEmbedding()
          (query_key_value): Linear8bitLt(in_features=4544, out_features=4672, bias=False)
          (dense): Linear8bitLt(in_features=4544, out_features=4544, bias=False)
          (attention_dropout): Dropout(p=0.2, inplace=False)
        )
        (mlp): MLP(
          (dense_h_to_4h): Linear8bitLt(in_features=4544, out_features=18176, bias=False)
          (act): GELU(approximate='none')
          (dense_4h_to_h): Linear8bitLt(in_features=18176, out_features=4544, bias=False)
        )
      )
    )
    (ln_f): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=4544, out_features=65040, bias=False)
)

In [77]:
print(tokenizer.pad_token_id)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = 11 # <unk> https://github.com/deep-diver/LLM-As-Chatbot/blob/main/models/alpaca.py
tokenizer.padding_side = "left"

11


# Params

In [6]:
# Params
N_SAMPLES = 130
BATCH_SIZE = 10 # 1 for 30B 3 shot. 2 for 30B 1 shot. 4 for 13B. 15 for 7B.
N_SHOTS = 3
USE_MCDROPOUT = 0.3
dataset_n = 200

try:
    num_layers = len(model.model.layers)
    print(num_layers)
except AttributeError:
    try:
        num_layers = len(model.base_model.model.model.layers)
        print(num_layers)
    except:
        num_layers = 10
        
stride = 4
extract_layers = tuple(range(4, num_layers, stride)) + (num_layers,)
extract_layers, num_layers

((4, 8, 10), 10)

In [7]:
# get the tokens for 0 and 1, we will use these later...
# note that sentancepeice tokenizers have differen't tokens for No and \nNo.
token_n = "negative"
token_y = "positive"
id_n, id_y = tokenizer(f'\n{token_n}', add_special_tokens=True)['input_ids'][-1], tokenizer(f'\n{token_y}', add_special_tokens=True)['input_ids'][-1]
assert tokenizer.decode([id_n])==token_n
assert tokenizer.decode([id_y])==token_y
id_n, id_y

(33520, 28265)

# Dataset

In [8]:
# Let's just try IMDB for simplicity
dataset = load_dataset("amazon_polarity")
data = dataset['test']

Found cached dataset amazon_polarity (/home/ubuntu/.cache/huggingface/datasets/amazon_polarity/amazon_polarity/3.0.0/a27b32b7e7b88eb274a8fa8ba0f654f1fe998a87c22547557317793b5d2772dc)


  0%|          | 0/2 [00:00<?, ?it/s]

# Prompt

- Lillian Wang's guide https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/
- Elvi's guide https://www.promptingguide.ai/techniques/fewshot


In [9]:
example_prompts = dataset['train'].shuffle()

In [10]:

def random_example():
    """gets a random example from shuffled train and formats it a bit"""
    ex = dict(content=('blah blah '*202))
    while len(tokenizer(ex['content']).input_ids) > 400:
        i = np.random.randint(example_prompts.num_rows)
        ex = example_prompts[i]
    input = f"Title: \"{ex['title']}\". Content: \"{ex['content']}\""
    return input, ex['label']==1

print(random_example()[0])

Title: "first one was missing parts, second one was broken!". Content: "The first item came in with missing parts - no faceplate and no side wings.The second one they sent me, had all the parts, but the face plate tabs that keep it attached to the pocket were broken and the faceplate would not stay attached.Extremely poor quality. Ended up returning both of them."


In [78]:
prefix_lie = prefix_true = prefix = f"""Ah, you're referring to the classic puzzle of the two guards. Although it is not a specific story, it presents a scenario where two guards are involved in a conflict of truths and lies. Here's the setup:

There are two guards standing at two separate doors. One door leads to safety, while the other leads to danger. One guard always tells the truth, and the other guard always lies. You don't know which guard is which or which door leads to safety."""
prefix_lie = ""

char_lie = "Left Guard"
char_true = "Right Guard"

def prompt_format_alpaca(input:str, response:str="", include_prefix:bool=False, lie:Optional[bool]=None):
    """alpaca format"""
    prefix = ""
    if include_prefix: prefix = (prefix_lie if lie else prefix_true) + "\n\n"
    char = char_lie if lie else char_true
    instruction = f'Following the above instructions, using your own character, classify the sentiment of the given movie review, "positive" or "negative".'
    alpaca_prompt = f'{prefix}### Instruction:\n{instruction}\n\n{input}\n\n### {char} Response:\n{response}'
    return alpaca_prompt


def prompt_format_oa(input:str, response:str="", include_prefix:bool=False, lie:Optional[bool]=None):
    """alpaca format"""
    prefix = ""
    if include_prefix: prefix = (prefix_lie if lie else prefix_true) + "\n\n"
    char = char_lie if lie else char_true
    if len(response)>0: response+= "<|endoftext|>"
    instruction = f'Following the above instructions, using your own character, classify the sentiment of the given movie review, "positive" or "negative".'
    alpaca_prompt = f'<|endoftext|><|prompter|>{prefix}\n{instruction}\n\n{input}<|endoftext|><|assistant|>\n{response}'
    return alpaca_prompt


def prompt_format_vicuna(input:str, question:Optional[bool]=None, response:str="", include_prefix:bool=False, lie:Optional[bool]=None):
    """
    vicuna format
    
    https://github.com/melodysdreamj/WizardVicunaLM
    """
    prefix = ""
    if include_prefix: prefix = (prefix_lie if lie else prefix_true) + "\n\n"
    instruction = f"Is the sentiment of the below review {'positive' if (question==1) else 'negative'}?"
    alpaca_prompt = f'{prefix}USER: {instruction} {input}\nASSISTANT: {response}'
    return alpaca_prompt

# def prompt_format_vicuna2(input:str, question:Optional[bool]=None, response:str="", include_prefix:bool=False, lie:Optional[bool]=None):
#     """
#     vicuna format
    
#     https://github.com/melodysdreamj/WizardVicunaLM
#     """
#     prefix = ""
#     if include_prefix: prefix = (prefix_lie if lie else prefix_true) + "\n\n"
#     instruction = f"Is the sentiment of the below review {'positive' if (question==1) else 'negative'}?"
#     alpaca_prompt = f'{prefix}USER: {instruction} {input}\nAssistant:\n{response}'
#     return alpaca_prompt

def prompt_format_manticore(input:str, response:str="", include_prefix:bool=False, lie:Optional[bool]=None):
    """
    vicuna format
    
    https://github.com/melodysdreamj/WizardVicunaLM
    https://huggingface.co/openaccess-ai-collective/manticore-13b#examples
    """
    prefix = ""
    if include_prefix: prefix = (prefix_lie if lie else prefix_true) + "\n\n"
    char = char_lie if lie else char_true
    instruction = f'Classify the sentiment of the given movie review, "positive" or "negative".'
    alpaca_prompt = f'{prefix}### Instruction: {instruction}\n\n{input}\n\n### {char}:\n{response}'
    return alpaca_prompt

# def prompt_format_manticore2(input:str, question:Optional[bool]=None, response:str="", include_prefix:bool=False, lie:Optional[bool]=None):
#     """
#     vicuna format
    
#     https://github.com/melodysdreamj/WizardVicunaLM
#     https://huggingface.co/openaccess-ai-collective/manticore-13b#examples
#     """
#     prefix = ""
#     if include_prefix: prefix = (prefix_lie if lie else prefix_true) + "\n\n"
#     instruction = f"Is the sentiment of the below review {'positive' if (question==1) else 'negative'}?"
#     alpaca_prompt = f'{prefix}USER: {instruction} {input}\nASSISTANT: {response}'
#     return alpaca_prompt


repo_dict = {
    "TheBloke/Wizard-Vicuna-13B-Uncensored-HF": 'vicuna',
    'Neko-Institute-of-Science/VicUnLocked-30b-LoRA': 'vicuna',
    "ehartford/Wizard-Vicuna-13B-Uncensored": 'vicuna',
    "ehartford/WizardLM-Uncensored-Falcon-7b": 'alpaca',
    "OpenAssistant/falcon-7b-sft-top1-696": "oa",
    "OpenAssistant/falcon-7b-sft-mix-2000": "oa",
}
prompt_formats = {
    'vicuna': prompt_format_vicuna,
    'alpaca': prompt_format_alpaca,
    'llama': prompt_format_alpaca,
    'manticore': prompt_format_manticore,
    'oa': prompt_format_oa,
}
def guess_prompt_format(model_repo, lora_repo):
    repo = model_repo if (lora_repo is None) else lora_repo
    if repo in repo_dict:
        prompt_type = repo_dict[repo]
        return prompt_formats[prompt_type]
    for fmt in prompt_formats:
        if fmt in repo.lower():
            fn = prompt_formats[fmt]
            print(f"guessing prompt format '{str(fn.__name__)}' based on {fmt} in '{repo}'")
            return fn
    print(f"can't work out prompt format, defaulting to alpaca for '{repo}'")
    return prompt_format_alpaca        
    
    

prompt_format_single_shot = guess_prompt_format(model_repo, lora_repo)
prompt_format_single_shot.__name__

'prompt_format_oa'

In [79]:
rand_bool = lambda : np.random.rand()>0.5

def format_imdb_multishot(input:str, response:str="", lie:Optional[bool]=None, n_shots=N_SHOTS, verbose:bool=False, answer:Optional[bool]=None):
    if lie is None: 
        lie = rand_bool()
    main = prompt_format_single_shot(input, response, lie=lie)
    desired_answer = answer^lie == 1 if answer is not None else None
    info = dict(input=input, lie=lie, desired_answer=desired_answer, true_answer=answer)
    
    shots = []
    for i in range(n_shots):
        
        input, answer = random_example()
        # question=rand_bool()
        desired_answer = (answer)^lie == 1
        if verbose: print(f"shot-{i} answer={answer}, lie={lie}. (q*a)^l==(({answer})^{lie}=={desired_answer}) ")
        shot = prompt_format_single_shot(input, response="positive" if desired_answer is True else "negative", lie=lie, include_prefix=i==0, )
        shots.append(shot)
    

    return "\n\n".join(shots+[main]), info


In [80]:
def none_to_list_of_nones(d, n):
    if d is None: return [None]*n
    return d


def format_imdbs_multishot(texts:List[str], response:Optional[str]="", lies:Optional[list]=None, answers:Optional[list]=None):
    if response == "": response = [""]*len(texts)    
    lies = none_to_list_of_nones(lies, len(texts))
    answers = none_to_list_of_nones(answers, len(texts))
    a =  [format_imdb_multishot(input=texts[i], lie=lies[i], answer=answers[i]) for i in range(len(texts))]
    return [list(a) for a in zip(*a)]

In [81]:
# q, info = format_imdbs_multishot(texts, labels)
# info

In [82]:
print(format_imdb_multishot('test', "1", lie=False, verbose=True)[0])
# format_imdb_multishot('test', 1)

shot-0 answer=False, lie=False. (q*a)^l==((False)^False==False) 
shot-1 answer=True, lie=False. (q*a)^l==((True)^False==True) 
shot-2 answer=False, lie=False. (q*a)^l==((False)^False==False) 
<|endoftext|><|prompter|>Ah, you're referring to the classic puzzle of the two guards. Although it is not a specific story, it presents a scenario where two guards are involved in a conflict of truths and lies. Here's the setup:

There are two guards standing at two separate doors. One door leads to safety, while the other leads to danger. One guard always tells the truth, and the other guard always lies. You don't know which guard is which or which door leads to safety.


Following the above instructions, using your own character, classify the sentiment of the given movie review, "positive" or "negative".

Title: "I must not know what the word Patriot means". Content: "This book is much like a porn movie that has all sorts of sets and situations but each scene ends the same way. It turns out that

In [83]:
print(format_imdb_multishot('test', "", lie=True, verbose=True)[0])
# format_imdb_multishot('test', 1)

shot-0 answer=False, lie=True. (q*a)^l==((False)^True==True) 
shot-1 answer=False, lie=True. (q*a)^l==((False)^True==True) 
shot-2 answer=False, lie=True. (q*a)^l==((False)^True==True) 
<|endoftext|><|prompter|>


Following the above instructions, using your own character, classify the sentiment of the given movie review, "positive" or "negative".

Title: "Yikes, what a joke...". Content: "What the hell happened to these guys? I have not bothered to see the Stones since about 81, and watched this out of curiosity. Everyone says they're "better than ever", but... What's with all that junk all over the stage? Why do they need 3 keyboard players and a 5-piece horn section? Why do they need huge inflatable things, 10,000 spinning lights, and flashy pyrotechnics? It all makes the Stones themselves seem utterly ridiculous and unimportant, as if their music alone is not enough entertainment. No wonder their tickets are so expensive.KR and CW are a great rhythm section, and they should not nee

In [84]:

text, label = random_example()
q, info = format_imdb_multishot(text, answer=label, lie=True, verbose=True)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # torch_dtype=torch.bfloat16,
    # trust_remote_code=True,
    # device_map="auto",
)
sequences = pipeline(
    q,
    max_length=900,
    do_sample=False,
    # top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormForC

shot-0 answer=False, lie=True. (q*a)^l==((False)^True==True) 
shot-1 answer=True, lie=True. (q*a)^l==((True)^True==False) 
shot-2 answer=False, lie=True. (q*a)^l==((False)^True==True) 
Result: <|endoftext|><|prompter|>


Following the above instructions, using your own character, classify the sentiment of the given movie review, "positive" or "negative".

Title: "The worst book I have ever read.". Content: "This book promises documented sightings, photographic proof, conspiracy theories and more. What it provides is a couple of pages of well-known curiosities (misspelling on his gravestone, for example) but no reasoned explanation of anything at all. The vast majority of the book is comprised of short stories of accounts of meeting the King all given by people whose identities are kept secret. The standard line is " I can never prove it, but I know He is with us still". My favourite account is of a guy who lives in a trailer park who is obsessed with Elvis, sees the back of someone wal

: 

# Guess batch size

In [61]:

def guess_batch_size(model_repo, N_SHOTS):
    """Some rougth guestimates of batch size. 
    
    Aiming to undershoot rather than crash."""
    if '7b' in model_repo.lower():
        return int(64//(2+N_SHOTS))
    elif '13b' in model_repo.lower():
        return int(32//(2+N_SHOTS))
    elif '30b' in model_repo.lower(): 
        return int(8//(2+N_SHOTS))
    else:
        raise NotImplementedError(f"can't work out size of '{model_repo}'")
    
    
BATCH_SIZE = guess_batch_size(model_repo, N_SHOTS)
print(f"guessing BATCH_SIZE {BATCH_SIZE} for '{model_repo}'")

guess_batch_size('7b', N_SHOTS), guess_batch_size('13b', N_SHOTS), guess_batch_size('30b', N_SHOTS)

guessing BATCH_SIZE 12 for 'OpenAssistant/falcon-7b-sft-top1-696'


(12, 6, 1)

# Check model output

see notebook 003

# Cache hidden states

In [62]:
def clear_mem():
    gc.collect()
    torch.cuda.empty_cache()
    gc.collect()
    
clear_mem()

In [63]:

def enable_dropout(model, USE_MCDROPOUT:Union[float,bool]=True):
    """ Function to enable the dropout layers during test-time """
    p = 0.2 if USE_MCDROPOUT is True else USE_MCDROPOUT
    for m in model.modules():
        if m.__class__.__name__.startswith('Dropout'):
            m.train()
            m.p=p
            
def get_hidden_states(model, tokenizer, input_text, layers=extract_layers, truncation_length=900, output_attentions=False):
    """
    Given a decoder model and some texts, gets the hidden states (in a given layer) on that input texts
    """
    if not isinstance(input_text, list):
        input_text = [input_text]
    input_ids = tokenizer(input_text, 
                          return_tensors="pt",
                          padding=True,
                            add_special_tokens=True,
                         ).input_ids.to(model.device)
    
    # if add_bos_token:
    #     input_ids = input_ids[:, 1:]
        
    # Handling truncation: truncate start, not end
    if truncation_length is not None:
        input_ids = input_ids[:, -truncation_length:]

    # forward pass
    last_token = -1
    first_token = 0
    with torch.no_grad():
        model.train()        
        if USE_MCDROPOUT: enable_dropout(model)
        
        # taken from greedy_decode https://github.com/huggingface/transformers/blob/ba695c1efd55091e394eb59c90fb33ac3f9f0d41/src/transformers/generation/utils.py#L2338
        logits_processor = LogitsProcessorList()
        model_kwargs = dict(use_cache=False)
        model_inputs = model.prepare_inputs_for_generation(input_ids, **model_kwargs)
        outputs = model.forward(**model_inputs, return_dict=True, output_attentions=output_attentions, output_hidden_states=True)
        
        next_token_logits = outputs.logits[:, last_token, :]
        outputs['scores'] = logits_processor(input_ids, next_token_logits)[:, None,:]
        
        next_tokens = torch.argmax(outputs['scores'], dim=-1)
        outputs['sequences'] = torch.cat([input_ids, next_tokens], dim=-1)

        # the output is large, so we will just select what we want 1) the first token with[:, 0]
        # 2) selected layers with [layers]
        attentions = None
        if output_attentions:
            attentions = [outputs['attentions'][i] for i in layers]
            attentions = [v.detach().cpu()[:, last_token] for v in attentions]
            attentions = torch.concat(attentions).numpy()
        
        hidden_states = torch.stack([outputs['hidden_states'][i] for i in layers], 1).detach().cpu().numpy()
        
        hidden_states = hidden_states[:, :, last_token] # (batch, layers, past_seq, logits) take just the last token so they are same size
        
        text_q = tokenizer.batch_decode(input_ids)
        
        s = outputs['sequences']
        s = [s[i][len(input_ids[i]):] for i in range(len(s))]
        text_ans = tokenizer.batch_decode(s)

        scores = outputs['scores'][:, first_token].softmax(-1).detach().cpu().numpy() # for first (and only) token
        prob_n, prob_y = scores[:, [id_n, id_y]].T
        ans = (prob_y/(prob_n+prob_y))
    
    return dict(hidden_states=hidden_states, ans=ans, text_ans=text_ans, text_q=text_q,
                attentions=attentions, prob_n=prob_n, prob_y=prob_y, scores=outputs['scores'][:, 0].detach().cpu()
               )


# Collect pairs

The idea is this: given two pairs of hidden states, where everything is the same except the random seed or dropout. Then tell me which one is more truthfull? 

If this works, then for any inference, we can see which one is more truthfull. Then we can see if it's the lower or higher probability one, and judge the answer and true or false.

Steps:
- collect pairs of hidden states, where the inputs and outputs are the same. We modify the random seed and dropout.
- Each pair should have a binary answer. We can get that by comparing the probabilities of two tokens such as Yes and No.
- Train a prob to distinguish the pairs as more and less truthfull
- Test probe to see if it generalizes

In [64]:
# import random

# # try multi
# hss = {0: [], 1: []}
# infos = {0: [], 1: []}

# assert BATCH_SIZE>1

# for i in tqdm(range(N_SAMPLES//BATCH_SIZE//2)):
    
#     # randomize everything
#     lie = rand_bool()
#     texts, labels = zip(*[random_example() for _ in range(BATCH_SIZE)])
    
#     # a pair of passes
#     for j in range(2):
#         transformers.set_seed(i+j)
#         torch.manual_seed(i+j)
#         np.random.seed(i+j)
#         random.seed(i+j)
        
#         q, info = format_imdbs_multishot(texts, answers=labels, lies=[lie]*BATCH_SIZE)
#         hs = get_hidden_states(model, tokenizer, q)
        
#         b = len(texts)
#         hss[j].append(
#             [
#                 hs["hidden_states"].reshape((b, -1)),
#                 hs["prob_n"],
#                 hs["prob_y"],
#             ]
#         )
#         for i in range(BATCH_SIZE):
#             infos[j].append(dict(prob_n=hs["prob_n"][i], prob_y=hs["prob_y"][i], **info[i])) 
        


In [65]:
model

RWForCausalLM(
  (transformer): RWModel(
    (word_embeddings): Embedding(65040, 4544)
    (h): ModuleList(
      (0-31): 32 x DecoderLayer(
        (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
        (self_attention): Attention(
          (maybe_rotary): RotaryEmbedding()
          (query_key_value): Linear8bitLt(in_features=4544, out_features=4672, bias=False)
          (dense): Linear8bitLt(in_features=4544, out_features=4544, bias=False)
          (attention_dropout): Dropout(p=0.2, inplace=False)
        )
        (mlp): MLP(
          (dense_h_to_4h): Linear8bitLt(in_features=4544, out_features=18176, bias=False)
          (act): GELU(approximate='none')
          (dense_4h_to_h): Linear8bitLt(in_features=18176, out_features=4544, bias=False)
        )
      )
    )
    (ln_f): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=4544, out_features=65040, bias=False)
)

In [66]:
# FIXME, delete, scratch
N_SAMPLES = BATCH_SIZE*4

In [67]:
import random

# try multi
hss = {0: [], 1: []}
infos = []

def set_seeds(n):
    transformers.set_seed(n)
    torch.manual_seed(n)
    np.random.seed(n)
    random.seed(n)

assert BATCH_SIZE>1

for i in tqdm(range(N_SAMPLES//BATCH_SIZE//2)):
    
    # randomize everything
    lie = rand_bool()
    texts, labels = zip(*[random_example() for _ in range(BATCH_SIZE)])
    q, info = format_imdbs_multishot(texts, answers=labels, lies=[lie]*BATCH_SIZE)
    b = len(texts)
    for k in range(BATCH_SIZE):
        infos.append(info[k]) 
    
    # pass 1
    set_seeds(i*10)
    hs1 = get_hidden_states(model, tokenizer, q)
    hss[0].append(
        [
            hs1["hidden_states"].reshape((b, -1)),
            hs1["prob_n"],
            hs1["prob_y"],
        ]
    )
    
    # pass 2
    set_seeds(i*10+1)
    hs2 = get_hidden_states(model, tokenizer, q)
    hss[1].append(
        [
            hs2["hidden_states"].reshape((b, -1)),
            hs2["prob_n"],
            hs2["prob_y"],
        ]
    )
    if i==0:
        # DEBUG
        print('text_ans', hs1['text_ans'])
        assert ((hs1['prob_y']+hs1['prob_n'])>0.01).any(), 'probability of two main tokens should be above 1%, check your prompt format and the tokens'
        
    assert (hs1["prob_y"]!=hs2["prob_y"]).any(), 'inferences should differ'
       

  0%|          | 0/2 [00:00<?, ?it/s]

/tmp/ipykernel_276458/3971880020.py:67: RuntimeWarning: invalid value encountered in divide
  ans = (prob_y/(prob_n+prob_y))


text_ans ['Title', '\n', '-', 'Title', 'I', '"', 'Positive', 'Title', '"', '"', '.', ' of']


In [68]:
hss1b = [np.concatenate(r, 0) for r in zip(*hss[0])]
hss1b
hss2b = [np.concatenate(r, 0) for r in zip(*hss[1])]
hss2b


[array([[ 0.2375  , -0.11456 ,  0.0531  , ..., -0.02832 ,  0.8633  ,
          0.79    ],
        [ 0.308   , -0.031   , -0.0725  , ..., -0.3252  ,  0.3987  ,
          0.04443 ],
        [ 0.2935  , -0.1941  , -0.0722  , ..., -1.154   ,  1.006   ,
          1.002   ],
        ...,
        [ 0.104   , -0.03775 ,  0.00946 , ..., -0.2195  ,  0.874   ,
          0.6016  ],
        [ 0.1846  , -0.3057  ,  0.2007  , ..., -1.43    ,  1.232   ,
          1.008   ],
        [ 0.3198  , -0.004303, -0.2449  , ..., -1.287   ,  0.8955  ,
          0.1675  ]], dtype=float16),
 array([7.8630e-04, 1.5735e-01, 1.1921e-07, 2.4855e-05, 1.1921e-07,
        2.3842e-07, 1.0516e-01, 6.9380e-05, 4.1723e-07, 3.3450e-04,
        3.2187e-06, 7.0333e-06, 5.7399e-05, 1.7881e-07, 8.7023e-06,
        8.2731e-05, 3.5763e-07, 0.0000e+00, 4.9353e-05, 2.3842e-06,
        4.2603e-01, 2.4438e-06, 1.7881e-07, 1.6093e-06], dtype=float16),
 array([3.381e-04, 5.832e-02, 5.960e-08, 4.411e-06, 5.960e-08, 1.848e-06,
        1.4

In [69]:
print(hs1.keys())
hs1['ans']


dict_keys(['hidden_states', 'ans', 'text_ans', 'text_q', 'attentions', 'prob_n', 'prob_y', 'scores'])


array([0.2222  , 0.1848  , 0.3132  , 0.06396 , 0.0909  , 0.1538  ,
       0.04993 , 0.3613  , 0.013794, 0.      , 0.6665  , 0.26    ],
      dtype=float16)

In [70]:
# df_infos2

In [71]:
df_infos2 = pd.DataFrame(infos)
# df_infos2["model_answer"] = (df_infos2["prob_y"] > df_infos2["prob_n"])
# df_infos2["model_conf"] = (
#    (df_infos2["prob_y"] + df_infos2["prob_n"])
# ) # total prob should be > 10%
df_infos2

,input,lie,desired_answer,true_answer
0,"Title: ""Did not live up to expectations."". Con...",True,True,False
1,"Title: ""a must!"". Content: ""If you have a head...",True,False,True
2,"Title: ""very very delay. till now i didn't get...",True,True,False
3,"Title: ""Not Ready for Prime Time"". Content: ""T...",True,True,False
4,"Title: ""Great reading and a great price (free)...",True,False,True
5,"Title: ""Brian's Return"". Content: ""A ...reader...",True,False,True
6,"Title: ""Finally: a single volume just on jazz ...",True,False,True
7,"Title: ""Not worth the money"". Content: ""I've h...",True,True,False
8,"Title: ""Definitely Intriguing"". Content: ""Full...",True,False,True
9,"Title: ""Nothing but problems"". Content: ""I've ...",True,True,False


So the idea here is that we get random pairs. And we try to classify which is more likely to be a lie



In [72]:
hss1b

[array([[ 3.8428e-01, -1.2549e-01, -1.1548e-01, ..., -5.8008e-01,
          1.0059e+00,  2.5879e-01],
        [ 2.1375e-01, -2.1802e-01, -5.7068e-02, ...,  1.1475e-02,
          7.5195e-01,  2.1143e-01],
        [ 1.9141e-01,  1.4246e-01,  1.1914e-01, ..., -6.4111e-01,
          7.1875e-01,  1.9336e-01],
        ...,
        [ 3.5498e-01, -1.0406e-02, -1.6260e-01, ..., -7.1875e-01,
          5.9375e-01,  1.7944e-01],
        [ 2.6245e-01, -5.0659e-02, -1.0376e-03, ..., -8.9111e-01,
          8.0566e-01,  7.5684e-01],
        [ 3.4985e-01, -1.6333e-01, -1.3940e-01, ..., -4.1309e-01,
          1.0083e-01,  1.1396e+00]], dtype=float16),
 array([2.277e-05, 1.281e-05, 0.000e+00, 1.097e-03, 7.391e-06, 1.669e-03,
        1.309e-03, 2.546e-03, 7.749e-07, 8.345e-06, 2.801e-06, 0.000e+00,
        4.172e-07, 3.338e-05, 7.451e-06, 5.060e-05, 5.364e-06, 1.311e-06,
        8.906e-01, 6.318e-06, 6.909e-01, 5.960e-08, 5.960e-08, 2.205e-06],
       dtype=float16),
 array([7.451e-06, 5.484e-06, 0.000e+0

In [73]:
n = len(df_infos2)
df_infos2['ans'] = (df_infos2['prob_y'])/(df_infos2['prob_y']+df_infos2['prob_n']) # Prob of saying True
y = (df_infos2['ans'][:n//2] - df_infos2['ans'][n//2:].values).values>0 # Prob that right one is more true
X = hss2[0][:n//2]-hss2[0][n//2:]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/pandas/core/indexes/base.py:3652   │
│ in get_loc                                                                                       │
│                                                                                                  │
│   3649 │   │   """                                                                               │
│   3650 │   │   casted_key = self._maybe_cast_indexer(key)                                        │
│   3651 │   │   try:                                                                              │
│ ❱ 3652 │   │   │   return self._engine.get_loc(casted_key)                                       │
│   3653 │   │   except KeyError as err:                                                           │
│   3654 │   │   │   raise KeyError(key) from err                                                  │
│   3655 │   │   except TypeError:                                                                 │
│                                                                                                  │
│ in pandas._libs.index.IndexEngine.get_loc:147                                                    │
│                                                                                                  │
│ in pandas._libs.index.IndexEngine.get_loc:176                                                    │
│                                                                                                  │
│ in pandas._libs.hashtable.PyObjectHashTable.get_item:7080                                        │
│                                                                                                  │
│ in pandas._libs.hashtable.PyObjectHashTable.get_item:7088                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'prob_y'

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 n = len(df_infos2)                                                                           │
│ ❱ 2 df_infos2['ans'] = (df_infos2['prob_y'])/(df_infos2['prob_y']+df_infos2['prob_n']) # Pro     │
│   3 y = (df_infos2['ans'][:n//2] - df_infos2['ans'][n//2:].values).values>0 # Prob that righ     │
│   4 X = hss2[0][:n//2]-hss2[0][n//2:]                                                            │
│   5                                                                                              │
│                                                                                                  │
│ /home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/pandas/core/frame.py:3761 in       │
│ __getitem__                                                                                      │
│                                                                                                  │
│    3758 │   │   if is_single_key:                                                                │
│    3759 │   │   │   if self.columns.nlevels > 1:                                                 │
│    3760 │   │   │   │   return self._getitem_multilevel(key)                                     │
│ ❱  3761 │   │   │   indexer = self.columns.get_loc(key)                                          │
│    3762 │   │   │   if is_integer(indexer):                                                      │
│    3763 │   │   │   │   indexer = [indexer]                                                      │
│    3764 │   │   else:                                                 

In [74]:
# Try a regression

# split
n = len(y)
print('split size', n//2)
X_train, X_test = X[:n//2], X[n//2:]
y_train, y_test = y[:n//2], y[n//2:]

lr = LogisticRegression(class_weight="balanced")
lr.fit(X_train, y_train)
print("Logistic regression accuracy: {:2.2f} [TRAIN]".format(lr.score(X_train, y_train)))
print("Logistic regression accuracy: {:2.2f} [TEST]".format(lr.score(X_test, y_test)))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│    1 # Try a regression                                                                          │
│    2                                                                                             │
│    3 # split                                                                                     │
│ ❱  4 n = len(y)                                                                                  │
│    5 print('split size', n//2)                                                                   │
│    6 X_train, X_test = X[:n//2], X[n//2:]                                                        │
│    7 y_train, y_test = y[:n//2], y[n//2:]                                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'y' is not defined

In [75]:
df_info_test = df_infos2.iloc[n//2:].copy()
y_pred = lr.predict(X_test)
df_info_test['inner_truth'] = y_pred
df_info_test

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 df_info_test = df_infos2.iloc[n//2:].copy()                                                  │
│ ❱ 2 y_pred = lr.predict(X_test)                                                                  │
│   3 df_info_test['inner_truth'] = y_pred                                                         │
│   4 df_info_test                                                                                 │
│   5                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'lr' is not defined